In [34]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
df = pd.read_csv('Dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
#data = df[df['Market share'] != 0]
#data.head(20)
df.head(5)
print(f'shape: {df.shape}')

shape: (1197, 14)


# Creating dummies

In [36]:
# Copy the dataframe
df2 = df.copy()

In [37]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)



In [38]:
df2.head(5)

,ID,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Country,...,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020,Year_2021,Year_2022,Year_2023,China
0,1,0.0,Aiways,U5,400,0.0,201,34,SUV,CH,...,0,0,0,0,0,0,0,0,0,1
1,1,0.0,Aiways,U5,400,0.0,201,34,SUV,CH,...,0,0,0,0,0,0,0,0,0,1
2,1,0.0,Aiways,U5,400,0.0,201,34,SUV,CH,...,1,0,0,0,0,0,0,0,0,1
3,1,0.0,Aiways,U5,400,0.0,201,34,SUV,CH,...,0,1,0,0,0,0,0,0,0,1
4,1,0.0,Aiways,U5,400,0.0,201,34,SUV,CH,...,0,0,1,0,0,0,0,0,0,1


# Creating log market share

In [39]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

c:\Users\Mikkel\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Independent OLS

In [40]:
data = df2
N = df['ID'].unique().size
T = df['Year'].unique().size
print(f' N = {N}, T = {T}')
print(f'shapes df2: {df2.shape} \n  shape df: {df.shape}')

y = df2['log_market_share']
print(f' y shape: {y.shape}')
x = df2[['const', 'Range', 'Price', 'HP', 'Chargetime']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]

#ones = np.ones(N*T, 1)
X = np.hstack([x, dummies])
#X = pd.concat([x, dummies], axis=1)

#xnames = X.columns
#xnames = xnames.tolist()


assert df.shape[0] == N*T, f'Error: data is not a balanced panel'
print(f'Data has N={N} and T={T} data is = {df2.shape[0]}, {N*T}')

 N = 109, T = 11
shapes df2: (1197, 31) 
  shape df: (1197, 14)
 y shape: (1197,)


AssertionError: Error: data is not a balanced panel

In [8]:
OLS_model = sm.OLS(y, X)
OLS_result = OLS_model.fit()
# Print summary of the regression results
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:       log_market_share   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     14.19
Date:                Thu, 07 Mar 2024   Prob (F-statistic):           4.00e-35
Time:                        15:10:47   Log-Likelihood:                -639.15
No. Observations:                 334   AIC:                             1324.
Df Residuals:                     311   BIC:                             1412.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6280      0.955     -3.800      0.0

In [9]:
coefficients = OLS_result.params
print(f'coefficients: {coefficients}')
coefficients_labels = xnames[1:]

model_labels = data['Model'].unique()
j = 0
for i in model_labels:
    j = j
    #print(f'Car {j}: {i}')
    j = j+1

#X skal være [i:i+1,2:] 2: for at fjerne consten og pris
#beta skal være [2:] for at fjerne konstant og pris-koefficienten og reshapes for at kunne lave matrixmultiplikaiton. 

alpha = coefficients[1]
beta = coefficients
p_j = data['Price'].values#.reshape((N*T,1))
print(f'pt_j: {p_j.shape}')


coefficients: const        -3.628008e+00
Range         8.307278e-03
Price        -1.869419e-08
HP           -1.842563e-03
Chargetime   -3.447909e-02
Segment_B    -3.216900e-01
Segment_C     3.049201e-01
Segment_D    -7.320388e-02
Segment_E    -1.514931e+00
Segment_F    -1.772606e+00
Segment_M    -1.532133e+00
Segment_J    -5.085308e-01
Year_2014     4.829981e-01
Year_2015    -3.772373e-01
Year_2016     3.211447e-02
Year_2017    -1.754662e-01
Year_2018    -8.006394e-01
Year_2019    -1.211335e+00
Year_2020    -1.870289e+00
Year_2021    -2.874557e+00
Year_2022    -3.302745e+00
Year_2023    -3.927324e+00
China        -6.443649e-01
dtype: float64
pt_j: (334,)


In [10]:
fun.logit(alpha, beta, X, p_j, model_labels, coefficients_labels, coefficients)

InvalidIndexError: (slice(0, 1, None), slice(2, None, None))